In [1]:
import sys 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
import tensorflow as tf
sys.path.append('../deep_boltzmann')

C:\Users\Jeremy\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [3]:
rcParams.update({'font.size': 16})

In [4]:
# Switch AUTORELOAD ON. Disable this when in production mode!
%load_ext autoreload
%autoreload 2

In [5]:
from deep_boltzmann.models import ParticleDimer
from deep_boltzmann.networks.invertible import invnet, EnergyInvNet, create_RealNVPNet
from deep_boltzmann.sampling import GaussianPriorMCMC
from deep_boltzmann.sampling.latent_sampling import BiasedModel
from deep_boltzmann.sampling.permutation import HungarianMapper
from deep_boltzmann.util import load_obj, save_obj

ModuleNotFoundError: No module named 'deep_boltzmann'

In [ ]:
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite

In [ ]:
# reweighting
def test_sample_rew(network, rcfunc, rcmin, rcmax, temperature=1.0, nsample=100000):
    sample_z, sample_x, energy_z, energy_x, log_w = network.sample(temperature=1.0, nsample=nsample)
    bin_means, Es = free_energy_bootstrap(rcfunc(sample_x), rcmin, rcmax, 100, sample=100, weights=np.exp(log_w))
    fig = plt.figure(figsize=(5, 4))
    # model.plot_dimer_energy()
    plt.ylim(-10, 20)
    Emean = mean_finite(Es, axis=0)-7
    Estd = std_finite(Es, axis=0)
    plt.errorbar(bin_means, Emean, 2*Estd)
    # variance
    var = mean_finite(std_finite(Es, axis=0) ** 2)
    print('Estimator Standard Error: ', np.sqrt(var))
    return fig, bin_means, Emean, Estd

In [ ]:
def latent_interpolation(bg, x1, x2, nstep=1000, through_origin=False):
    lambdas = np.array([np.linspace(0, 1, num=nstep)]).T
    x1 = np.array([x1])
    x2 = np.array([x2])
    z1 = bg.transform_xz(x1)
    z2 = bg.transform_xz(x2)
    if through_origin:
        zpath1 = z1 * (1-lambdas[::2])
        zpath2 = z2 * (lambdas[::2]) 
        zpath = np.vstack([zpath1, zpath2])
    else:
        zpath = z1 + lambdas*(z2-z1)
    xpath = bg.transform_zx(zpath)
    return xpath

In [ ]:
def low_energy_fraction(energies, Emax):
    low_energy_count = [np.count_nonzero(E<Emax) for E in energies]
    sizes = [E.size for E in energies]
    low_energy_fraction = np.array(low_energy_count) / sizes
    return low_energy_fraction

In [ ]:
def plot_convergence(hist_ML, hist_KL, enerx_cut, enerz_cut, MLcol=1, KLcol=2):
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
    niter1 = len(hist_ML[0])
    niter2 = hist_KL[1].shape[0]
    niter = niter1 + niter2
    # ML loss
    losses_ML = np.concatenate([hist_ML[0], hist_KL[1][:, MLcol]])
    xticks = np.arange(niter1 + niter2) + 1
    axes[0].plot(xticks, losses_ML, color='black')
    axes[0].set_xlim(0, niter + 1)
    axes[0].set_ylabel('ML loss')
    axes[0].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # KL loss
    losses_KL = hist_KL[1][:, KLcol]
    xticks = np.arange(niter1, niter1 + niter2) + 1
    axes[1].plot(xticks, losses_KL, color='black')
    axes[1].set_xlim(0, niter + 1)
    axes[1].set_ylabel('KL loss')
    axes[1].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # low energy fractions
    enerx = hist_ML[2] + hist_KL[3]
    enerz = hist_ML[3] + hist_KL[4]
    lef_x = low_energy_fraction(enerx, enerx_cut)
    lef_z = low_energy_fraction(enerz, enerz_cut)
    axes[2].plot(lef_x, color='black', label='x')
    axes[2].plot(lef_z, color='blue', label='z')
    axes[2].set_xlim(0, niter + 1)
    axes[2].set_ylim(0, 1.05)
    axes[2].axvline(x=200, color='red', linestyle='--', linewidth=3)
    axes[2].set_ylabel('Training iterations')
    axes[2].set_ylabel('Low energy fraction')
    axes[2].legend()
    return fig, axes

In [ ]:
paper_dir = '/Users/noe/data/papers/NoeEtAl_BoltzmannGeneratorsRev/'

Particle model
---
Run notebook "Particles_Simulation_Data" to generate data

In [ ]:
# load trajectory data
trajdict = np.load('../data/particles_tilted/trajdata_long.npz')
import ast
params = ast.literal_eval(str(trajdict['params']))
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [ ]:
# create model
params['grid_k'] = 0.0
model = ParticleDimer(params=params)

In [ ]:
xx, xE = model.plot_dimer_energy();
plt.figure(figsize=(5, 2))
plt.plot(xx, xE, linewidth=2)
plt.ylim(-3, 22)
plt.xlabel('Dimer distance / nm')
plt.ylabel('Energy / $kT_0$')
#plt.savefig(paper_dir + 'figs/particles/particle_dimer_potential.pdf', bbox_inches='tight')

In [ ]:
W = np.exp(-xE)
-np.log(np.sum(W[xx < 1.5]) / np.sum(W[xx >= 1.5]))

In [ ]:
# hyperparameters
hyperparams = {'layer_types' : 'RRRRRRRR',
               'nl_layers' : 3,
               'nl_hidden' : 200,
               'nl_activation' : 'tanh',
               'zstd' : 1.0,
               'reg_Jxz' : 0.0,
               'temperature' : [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 4.0],
               'weight_ML' : 0.1,
               'weight_W2' : 0.0,
               'weight_RC' : 10.0,
               }

Umbrella sampling reference
-------

In [ ]:
from deep_boltzmann.sampling.umbrella_sampling import UmbrellaSampling
from deep_boltzmann.sampling.metropolis import MetropolisGauss

In [ ]:
## Either run Umbrella Sampling here....
#sampler = MetropolisGauss(model, model.init_positions(1.0), noise=0.02, stride=10)
#sampler.run(nsteps=10000)
#us = UmbrellaSampling(model, sampler, model.dimer_distance, sampler.traj[-1], 30, 250, 0.7, 2.3, forward_backward=True)
#us.run(nsteps=20000, verbose=True)

In [ ]:
## ... or load saved Umbrella Sampling data.
npzfile = np.load('./particles/US_data.npz')
colors = ['black', 'purple', 'blue', 'orange', 'red']
x = npzfile['umbrella_positions']
temperatures_US = npzfile['temperatures']
dF_closed_open_US = []
for i, dF in enumerate(npzfile['umbrella_free_energies']):
    icolor = i // 3
    plt.plot(x, dF, color=colors[icolor])
    W = np.exp(-dF)
    dF_at_kT0 = -np.log(np.sum(W[x < 1.5]) / np.sum(W[x >= 1.5]))
    dF_closed_open_US.append(dF_at_kT0)
    print(temperatures_US[i], dF_at_kT0)    

In [ ]:
# Estimate free energy profile and errors at kT=1 
umbrella_positions = npzfile['umbrella_positions']
# split in two halves because we ran forward and backward
umbrella_positions = umbrella_positions[:umbrella_positions.size//2]
umbrella_free_energies = [npzfile['umbrella_free_energies'][i][:umbrella_positions.size] for i in range(9, 12)] \
                       + [npzfile['umbrella_free_energies'][i][umbrella_positions.size:][::-1] for i in range(9, 12)] 
# align values
umbrella_free_energies = [F-F.mean() for F in umbrella_free_energies]

Boltzmann Generator
-------

In [ ]:
batchsize_ML =  500
batchsize_KL = 1000
noise_intensity = 0.0

In [ ]:
X0 = np.vstack([traj_closed_train, traj_open_train])

In [ ]:
Nnoise = xval.shape[0]
X0noise = X0[np.random.choice(X0.shape[0], Nnoise)] + noise_intensity * np.random.randn(Nnoise, X0.shape[1])
X0noise = X0noise.astype(np.float32)

In [ ]:
#X0 = np.vstack([traj_closed_train[::100], traj_open_train[::100]])

In [ ]:
model.draw_config(X0noise[1001], dimercolor='blue', alpha=0.8);

In [ ]:
bg = invnet(model.dim, 'RRRRRRRR', energy_model=model, nl_layers=4, nl_hidden=200, #100
            nl_activation='relu', nl_activation_scale='tanh', whiten=X0noise)

In [ ]:
hist_bg_ML = bg.train_ML(X0noise, xval=xval, epochs=200, lr=0.001, batch_size=batchsize_ML, 
                         std=1.0, verbose=0, return_test_energies=True)

In [ ]:
plt.plot(hist_bg_ML[0])
plt.plot(hist_bg_ML[1])
#plt.ylim(0, 100)

In [ ]:
_, sample_x, _, energies_x, _ = bg.sample(nsample=20000)
sample_d = model.dimer_distance(sample_x)
plt.hist(sample_d[sample_d < 3], 100, log=True)
plt.xlim(0, 3)

Regular training
-----

In [ ]:
temperature=1.0

In [ ]:
# initial training
Eschedule = [[200,  0.00001, 1e6, 1e3,  0.0, 10.0],
             [100,  0.0001, 1e6,  300,  0.0, 10.0],
             [100,  0.0001, 1e5,  100,  0.0, 10.0],
             [100,  0.0001, 5e4,   50,  0.0, 10.0],
             [100,  0.0001, 5e4,   20,  0.0, 10.0],
             [200,  0.0001, 5e4,    5,  0.0, 10.0]]

In [ ]:
hists_bg_KL = []
for i, s in enumerate(Eschedule):
    print(s)#'high_energy =', s[0], 'weight_ML =', s[1], 'epochs =', s[2])
    sys.stdout.flush()
    hist_bg_KL = bg.train_flexible(X0noise, xval=xval, epochs=s[0], lr=s[1], batch_size=batchsize_KL,
                                   verbose=1, high_energy=s[2], max_energy=1e10,
                                   weight_ML=s[3], weight_KL=1.0, temperature=temperature, weight_MC=0.0, weight_W2=s[4],
                                   weight_RCEnt=s[5], rc_func=model.dimer_distance_tf, rc_min=0.5, rc_max=2.5,
                                   std=1.0, reg_Jxz=0.0, return_test_energies=True)
    hists_bg_KL.append(hist_bg_KL)

In [ ]:
xeners = []
zeners = []
for h in hists_bg_KL:
    xeners += h[3]
    zeners += h[4]
hist_bg_KL_combined = [hists_bg_KL[0][0], 
                       np.vstack([h[1] for h in hists_bg_KL]),
                       np.vstack([h[2] for h in hists_bg_KL]),
                       xeners, zeners]

In [ ]:
def plot_convergence(hist_ML, hist_KL, enerx_cut, enerz_cut, MLcol=1, KLcol=2):
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(5, 10))
    niter1 = len(hist_ML[0])
    niter2 = hist_KL[1].shape[0]
    niter = niter1 + niter2
    # ML loss
    losses_ML = np.concatenate([hist_ML[0], hist_KL[1][:, MLcol]])
    xticks = np.arange(niter1 + niter2) + 1
    axes[0].plot(xticks, losses_ML, color='black')
    axes[0].set_xlim(0, niter + 1)
    axes[0].set_ylabel('ML loss')
    axes[0].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # KL loss
    losses_KL = hist_KL[1][:, KLcol]
    xticks = np.arange(niter1, niter1 + niter2) + 1
    axes[1].plot(xticks, losses_KL, color='black')
    axes[1].set_xlim(0, niter + 1)
    axes[1].set_ylabel('KL loss')
    axes[1].axvline(x=200, color='red', linestyle='--', linewidth=3)
    # low energy fractions
    enerx = hist_ML[2] + hist_KL[3]
    enerz = hist_ML[3] + hist_KL[4]
    lef_x = low_energy_fraction(enerx, enerx_cut)
    lef_z = low_energy_fraction(enerz, enerz_cut)
    axes[2].plot(lef_x, color='black', label='x')
    axes[2].plot(lef_z, color='blue', label='z')
    axes[2].set_xlim(0, niter + 1)
    axes[2].set_ylim(0, 1.05)
    axes[2].axvline(x=200, color='red', linestyle='--', linewidth=3)
    axes[2].set_ylabel('Training iterations')
    axes[2].set_ylabel('Low energy fraction')
    axes[2].legend()
    return fig, axes

In [ ]:
def energy_cut_z(ndim, nstd=3):
    z = np.random.randn(10000, ndim)
    zener = 0.5 * np.sum(z**2, axis=1)
    #return zener
    std = np.sqrt(np.mean((zener - zener.mean())**2))
    return zener.mean() + nstd*std

In [ ]:
zener = energy_cut_z(model.dim, nstd=3)
#plt.hist(zener, 100);

In [ ]:
xcut = 80
zcut = energy_cut_z(model.dim, nstd=3)
print('zcut = ', zcut)
#hist_bg_KL_arr = np.vstack([hist_bg_KL[1] for hist_bg_KL in hists_bg_KL])
fig, axes = plot_convergence(hist_bg_ML, hist_bg_KL_combined, xcut, zcut, MLcol=1, KLcol=2);
axes[1].semilogy()
#plt.savefig(paper_dir + 'figs/particles/training_convergence.pdf', bbox_inches='tight', transparent=True)

In [ ]:
_, sample_x, _, energies_x, _ = bg.sample(nsample=20000)
sample_d = model.dimer_distance(sample_x)
plt.hist(sample_d[sample_d < 3], 100, log=True)
plt.xlim(0, 3)

In [ ]:
plt.hist(energies_x[np.bitwise_and(energies_x<100, sample_d<1.5)], 100, color='yellow', density=True);
plt.hist(energies_x[np.bitwise_and(energies_x<100, sample_d>1.5)], 100, color='green', density=True);
plt.hist(model.energy(traj_open_train), 100, color='blue', alpha=0.5, density=True);
plt.hist(model.energy(traj_closed_train), 100, color='red', alpha=0.5, density=True);

In [ ]:
model.draw_config(sample_x[1], dimercolor='blue', alpha=0.8);

In [ ]:
# show minimum energy path
Emin = 1e9
bestpath = None
for i in range(90):
    for j in range(90):
        path = latent_interpolation(bg, traj_closed_train[i*100], traj_open_train[j*100], nstep=9)
        E = model.energy(path).max()
        if E < Emin:
            Emin = E
            bestpath = path

In [ ]:
# Compressed version for paper
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(6.5, 10))
model.draw_config(bestpath[0], axis=axes[0, 0], dimercolor='blue', alpha=0.8);
model.draw_config(bestpath[2], axis=axes[1, 0], dimercolor='blue', alpha=0.8);
model.draw_config(bestpath[3], axis=axes[2, 0], dimercolor='orange', alpha=0.8);
model.draw_config(bestpath[4], axis=axes[2, 1], dimercolor='orange', alpha=0.8);
model.draw_config(bestpath[5], axis=axes[1, 1], dimercolor='red', alpha=0.8);
model.draw_config(bestpath[7], axis=axes[0, 1], dimercolor='red', alpha=0.8);
plt.subplots_adjust(wspace=0.03, hspace=0.03)
plt.savefig(paper_dir + 'figs/particles/interpolate.pdf', bbox_inches='tight')

Free energy calculation
--------
For the paper, we run estimation at a series of different temperatures and obtain estimates of the free energy profile by averaging 10 models. Here, we only make estimates at one temperature and use a faster averaging approach that has higher statistical error: We take only one model and sample it multiple times while training it in the already converged stage. We still get slightly different estimates for different samples because of the stochasticity in SGD, but the estimates are not statistically independent and thus lead to higher uncertainties than averaging independent models.

In [ ]:
from deep_boltzmann.networks.training import FlexibleTrainer
from deep_boltzmann.sampling.analysis import free_energy_bootstrap

In [ ]:
s = Eschedule[-1]
flexible_trainer = FlexibleTrainer(bg, lr=s[1], batch_size=batchsize_KL, high_energy=s[2], max_energy=1e10,
                                   std=1.0, temperature=temperature, w_KL=1.0, w_ML=s[3], 
                                   w_RC=s[5], rc_func=model.dimer_distance_tf, rc_min=0.5, rc_max=2.5)

In [ ]:
logws = []
sample_ds = []

In [ ]:
for i in range(20):
    print('\nIter:',i)
    # train a bit
    flexible_trainer.train(X0noise, epochs=10, verbose=1)
    # sample while training:
    _, sample_x, _, energies_x, logw = bg.sample(nsample=100000)
    logws.append(logw - logw.max())
    sample_ds.append(model.dimer_distance(sample_x))

In [ ]:
bin_means = None
sample_Fs = []
for sample_d, logw in zip(sample_ds, logws):
    bin_means, sample_F = free_energy_bootstrap(sample_d, bins=100, range=(0.8, 2.2), log_weights=logw, 
                                                nbootstrap=1000, align_bins=np.arange(100))
    sample_Fs.append(sample_F)

In [ ]:
plt.errorbar(bin_means, np.mean(np.vstack(sample_Fs), axis=0), np.std(np.vstack(sample_Fs), axis=0), color='green')
plt.errorbar(umbrella_positions, np.mean(umbrella_free_energies, axis=0)-1, np.std(umbrella_free_energies, axis=0),
             linewidth=3, color='black')
plt.ylim(-18, 15)

Test invertibility
------

In [ ]:
# test invertibility
z = np.random.randn(1000, bg.dim)
x, Jzx = bg.transform_zxJ(z)
zrec, Jxz = bg.transform_xzJ(x)

# invertible?
print(np.abs(z[:,:-6] - zrec[:,:-6]).max())

# Jacobian consistent?
print(np.abs(Jxz + Jzx).max())

Display sampling
------

In [ ]:
_, sample_x, _, energies_x, _ = bg.sample(nsample=100000)

In [ ]:
traj_closed_train_ener = model.energy(traj_closed_train)
traj_open_train_ener = model.energy(traj_open_train)

In [ ]:
I_closed = np.where(model.dimer_distance(sample_x) < 1.3)[0]
energies_x_closed = energies_x[I_closed][energies_x[I_closed]<150]
I_ts = np.where(np.logical_and(model.dimer_distance(sample_x) > 1.3, model.dimer_distance(sample_x) < 1.7))[0]
energies_x_ts = energies_x[I_ts][energies_x[I_ts]<150]
I_open = np.where(model.dimer_distance(sample_x) > 1.7)[0]
energies_x_open = energies_x[I_open][energies_x[I_open]<150]

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(4, 8), sharex=True)
plt.subplots_adjust(wspace=0.03, hspace=0.03)

axes[0].hist(traj_closed_train_ener, 100, color='black', histtype='step', linewidth=2, density=True);
axes[0].hist(traj_closed_train_ener, 100, color='grey', alpha=0.5, density=True);
axes[0].hist(energies_x_closed, 100, color='blue', histtype='step', linewidth=2, density=True);
axes[0].hist(energies_x_closed, 100, color='blue', alpha=0.3, density=True);
axes[0].set_yticks([])
axes[0].set_ylim(0, 0.08)
axes[0].set_xlim(20, 150)

axes[1].hist(energies_x_ts, 100, color='orange', histtype='step', linewidth=2, density=True);
axes[1].hist(energies_x_ts, 100, color='orange', alpha=0.3, density=True);
axes[1].set_yticks([])
axes[1].set_ylim(0, 0.08)
axes[1].set_xlim(20, 150)

axes[2].hist(traj_open_train_ener, 100, color='black', histtype='step', linewidth=2, density=True);
axes[2].hist(traj_open_train_ener, 100, color='grey', alpha=0.5, density=True);
axes[2].hist(energies_x_open, 100, color='red', histtype='step', linewidth=2, density=True);
axes[2].hist(energies_x_open, 100, color='red', alpha=0.3, density=True);
axes[2].set_yticks([])
axes[2].set_xlim(20, 150)
axes[2].set_ylim(0, 0.08)
axes[2].set_xlabel('Energy / $kT_0$')
plt.savefig(paper_dir + 'figs/particles/sampled_energies.pdf', transparent=True)

Display free energy profiles obtained from model averaging
-----

In [ ]:
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite
from deep_boltzmann.util import load_obj, save_obj
from deep_boltzmann.sampling.analysis import mean_finite, std_finite
from deep_boltzmann.sampling.umbrella_sampling import UmbrellaSampling

In [ ]:
# Umbrella sampling - reference
us05 = UmbrellaSampling.load('../local/particles/US_old/us_T05_F.pkl')
us10 = UmbrellaSampling.load('../local/particles/US_old/us_T10_F.pkl')
us20 = UmbrellaSampling.load('../local/particles/US_old/us_T20_F.pkl')

In [ ]:
umbrella_positions = us10.umbrella_positions
pmf_us05 = us05.umbrella_free_energies()
pmf_us10 = us10.umbrella_free_energies()
pmf_us20 = us20.umbrella_free_energies()
pmf_uss = [pmf_us05, pmf_us10, pmf_us20]

In [ ]:
# run training + analysis scripts to get this file 
many_sampled_distances = load_obj('../local/particles/model_averaging_old/distances_sample.pkl')

In [ ]:
many_sampled_distances.keys()

In [ ]:
def mean_free_energy(Ds, Ws):
    E = []
    ndrop=0
    for D, W in zip(Ds, Ws):
        # sort by descending weight
        I = np.argsort(W)[::-1]
        D_sorted = D[I][ndrop:]
        W_sorted = W[I][ndrop:]

        bins = np.linspace(0.5, 2.5, 30)
        bin_means = 0.5*(bins[:-1] + bins[1:])
        hist, _ = np.histogram(D_sorted, bins=bins, weights=np.exp(W_sorted))
        e = -np.log(hist)
        e -= np.concatenate([e[3:10],e[-10:-3]]).mean()
        E.append(e)
    E = np.array(E)
    return bin_means, mean_finite(E, axis=0, min_finite=2), std_finite(E, axis=0, min_finite=2)

In [ ]:
def cut_energy(mE, cut=35.0):
    mEmin = mE[np.isfinite(mE)].min()
    mE = np.where(mE-mEmin < cut, mE, np.nan)
    return mE

In [ ]:
bm05, mE05, sE05 = mean_free_energy(many_sampled_distances['D05'], many_sampled_distances['W05'])
mE05 = cut_energy(mE05, cut=35.0)
bm10, mE10, sE10 = mean_free_energy(many_sampled_distances['D10'], many_sampled_distances['W10'])
mE10 = cut_energy(mE10, cut=35.0)
bm20, mE20, sE20 = mean_free_energy(many_sampled_distances['D20'], many_sampled_distances['W20'])
mE20 = cut_energy(mE20, cut=35.0)

In [ ]:
plt.figure(figsize=(5, 5))
#plt.fill_between(bm, mE+20-1*sE, mE+20+1*sE, color='blue', alpha=0.3)
plt.fill_between(bm05, mE05+20-1*sE05, mE05+20+1*sE05, color='#008800', alpha=0.5)
plt.plot(us05.rc_discretization, us05.rc_free_energies+6.2, linewidth=4, color='black')
plt.plot(bm05, mE05+20, color='#008800', linewidth=0, marker='.', markersize=12, markeredgewidth=1, markeredgecolor='black', label='0.5 $T_0$')

plt.fill_between(bm10, mE10+8.5-1*sE10, mE10+8.5+1*sE10, color='#00BB00', alpha=0.5)
plt.plot(us10.rc_discretization, us10.rc_free_energies+0.5, linewidth=4, color='black')
plt.plot(bm10, mE10+8.5, color='#00BB00', linewidth=0, marker='.', markersize=12, markeredgewidth=1, markeredgecolor='black', label='1.0 $T_0$')

#plt.errorbar(bm20, mE20, sE20, color='#33FF00', linewidth=0, marker='.', markersize=8, elinewidth=2, label='2.0')
plt.fill_between(bm20, mE20-1*sE20, mE20+1*sE20, color='#00FF00', alpha=0.5)
plt.plot(us20.rc_discretization, us20.rc_free_energies-5.2, linewidth=4, color='black')
plt.plot(bm20, mE20, color='#00FF00', linewidth=0, marker='.', markersize=12, markeredgewidth=1, markeredgecolor='black', label='2.0 $T_0$')


plt.legend(loc='upper left', ncol=1, frameon=False, handletextpad=0, labelspacing=0)
plt.ylim(-10, 60)
plt.xlabel('Dimer distance / nm')
plt.ylabel('Free energy difference / $kT_0$', labelpad=-10)
plt.savefig(paper_dir + 'figs/particles/free_energies_temp2.pdf', bbox_inches='tight', transparent=True)